In [2]:
"""
write this in the Terminal:
conda install -c anaconda pandas-datareader

When asked:
The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates                                 pkgs/main --> anaconda
  certifi                                         pkgs/main --> anaconda
  openssl                                         pkgs/main --> anaconda
  qt                                              pkgs/main --> anaconda
Proceed ([y]/n)?

Press y

Install dash

jupyter nbextension enable --py widgetsnbextension

jupyter labextension install @jupyter-widgets/jupyterlab-manager

"""

import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import pandas as pd
import pandas_datareader as web
import numpy as np
import bs4 as bs
import pickle
import requests
import os
import pickle
import numpy as np
#import dash
#import dash_core_components as dcc
#import dash_html_components as html
from scipy.stats import norm # normal distribution
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
style.use("ggplot")

"""The purpose of the code below is to first get the tickers for each of the companites on the S&P500 index list, which we later on will
use to append all the historic data to the appropriate ticker. We use *requests.get* to access the data from Wikipedia, which is the list of tickers, after which we use *BeautifulSoup* to then read the HTML format of the website, and make a table. The *soup.find* command is used to specify where exactly in the HTML the table we want is located. 

We make an empty list, and use a for loop to read all entries in the column of the tickers and append each of these to the list we made (called tickers). 

We save it as a pickle as we want to access this data later on, and fast. We called this *"sp500tickers.pickle"*. """



'The purpose of the code below is to first get the tickers for each of the companites on the S&P500 index list, which we later on will\nuse to append all the historic data to the appropriate ticker. We use *requests.get* to access the data from Wikipedia, which is the list of tickers, after which we use *BeautifulSoup* to then read the HTML format of the website, and make a table. The *soup.find* command is used to specify where exactly in the HTML the table we want is located. \n\nWe make an empty list, and use a for loop to read all entries in the column of the tickers and append each of these to the list we made (called tickers). \n\nWe save it as a pickle as we want to access this data later on, and fast. We called this *"sp500tickers.pickle"*. '

The purpose of the code below is to first get the tickers for each of the companites on the S&P500 index list, which we later on will
use to append all the historic data to the appropriate ticker. We use *requests.get* to access the data from Wikipedia, which is the list of tickers, after which we use *BeautifulSoup* to then read the HTML format of the website, and make a table. The *soup.find* command is used to specify where exactly in the HTML the table we want is located. 

We make an empty list, and use a for loop to read all entries in the column of the tickers and append each of these to the list we made (called tickers). 

We save it as a pickle as we want to access this data later on, and fast. We called this *"sp500tickers.pickle"*. 



In [3]:
#Automating S&P500 - From Yahoo Finance - Close price adjusted for splits, and Adj. Close price is adjusted for both dividends and splits.
def save_sp500_tickers():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find("table", {"class": "wikitable sortable"})
    tickers = []
    for row in table.findAll("tr")[1:]:
        ticker = row.findAll("td")[0].text.replace(".","-")
        ticker = ticker[:-1]
        tickers.append(ticker)

    df_tickers = pd.DataFrame(tickers)
    df_tickers.to_csv("tickers.csv")

    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    
        print(tickers)

        return(tickers)
    
save_sp500_tickers()

df_tickers = pd.read_csv("tickers.csv", index_col= 0)
df_tickers.columns = ["Ticker"]
df_tickers.head()

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'ATO', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BHGE', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK-B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF-B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CPRI', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CE', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'COST', 'COTY', 'CCI', 'CS

,Ticker
0,MMM
1,ABT
2,ABBV
3,ABMD
4,ACN


Next, we need data for each ticker we have in the list. We get this from yahoo, and using the *DataReader* from pandas. 

Remember we saved the tickers as a pickle file, which we will now open and use to append the data to. Also, we make a separate folder for all of the CSV files we will get (505!). We define appropriate start and end dates for the data we want. 

We then create a for loop, and save each CSV file with price data in the stock_dfs folder. 

In [4]:

def get_names():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find("table", {"class": "wikitable sortable"})
    names = []
    for row in table.findAll("tr")[1:]:
        name = row.findAll("td")[1].text.replace(".","-")
        names.append(name)

    df_names = pd.DataFrame(names)
    df_names.to_csv("names.csv")

    print(names)

get_names()

df_names = pd.read_csv("names.csv", index_col= 0)
df_names.columns = ["Name"]
df_names.head()


def get_sectors():
    resp = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find("table", {"class": "wikitable sortable"})
    sectors = []
    for row in table.findAll("tr")[1:]:
        sector = row.findAll("td")[3].text.replace(".","-")
        sectors.append(sector)

    df_sectors = pd.DataFrame(sectors)
    df_sectors.to_csv("sectors.csv")

    print(sectors)

get_sectors()

df_sectors = pd.read_csv("sectors.csv", index_col= 0)
df_sectors.columns = ["Sector"]
df_sectors.head()

df_names_and_sector = df_names.join(df_sectors, how = "right")
df_names_and_sector.head()

df_tickers_names_sectors = df_tickers.join(df_names_and_sector, how = "right")
df_tickers_names_sectors.head()
#df_tickers_names_sectors.set_index("Ticker", inplace=True)
#df_tickers_names_sectors.head()
df_tickers_names_sectors.values.tolist

with open ("tickers_names_sectors.pickle", "wb") as f:
    pickle.dump(df_tickers_names_sectors, f)

['3M Company', 'Abbott Laboratories', 'AbbVie Inc-', 'ABIOMED Inc', 'Accenture plc', 'Activision Blizzard', 'Adobe Systems Inc', 'Advanced Micro Devices Inc', 'Advance Auto Parts', 'AES Corp', 'Affiliated Managers Group Inc', 'AFLAC Inc', 'Agilent Technologies Inc', 'Air Products & Chemicals Inc', 'Akamai Technologies Inc', 'Alaska Air Group Inc', 'Albemarle Corp', 'Alexandria Real Estate Equities', 'Alexion Pharmaceuticals', 'Align Technology', 'Allegion', 'Allergan, Plc', 'Alliance Data Systems', 'Alliant Energy Corp', 'Allstate Corp', 'Alphabet Inc Class A', 'Alphabet Inc Class C', 'Altria Group Inc', 'Amazon-com Inc-', 'Ameren Corp', 'American Airlines Group', 'American Electric Power', 'American Express Co', 'American International Group', 'American Tower Corp-', 'American Water Works Company Inc', 'Ameriprise Financial', 'AmerisourceBergen Corp', 'AMETEK Inc-', 'Amgen Inc-', 'Amphenol Corp', 'Anadarko Petroleum Corp', 'Analog Devices, Inc-', 'ANSYS', 'Anthem Inc-', 'Aon plc', 'A-

In [5]:
#Getting data from Yahoo
def data_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2016, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

data_yahoo()


Already have MMM
Already have ABT
Already have ABBV
Already have ABMD
Already have ACN
Already have ATVI
Already have ADBE
Already have AMD
Already have AAP
Already have AES
Already have AMG
Already have AFL
Already have A
Already have APD
Already have AKAM
Already have ALK
Already have ALB
Already have ARE
Already have ALXN
Already have ALGN
Already have ALLE
Already have AGN
Already have ADS
Already have LNT
Already have ALL
Already have GOOGL
Already have GOOG
Already have MO
Already have AMZN
Already have AEE
Already have AAL
Already have AEP
Already have AXP
Already have AIG
Already have AMT
Already have AWK
Already have AMP
Already have ABC
Already have AME
Already have AMGN
Already have APH
Already have APC
Already have ADI
Already have ANSS
Already have ANTM
Already have AON
Already have AOS
Already have APA
Already have AIV
Already have AAPL
Already have AMAT
Already have APTV
Already have ADM
Already have ARNC
Already have ANET
Already have AJG
Already have AIZ
Already have A

Furthermore, the data from yahoo and the tickers are not very useful by themselves, so obvisouly we want to compile the data in order to get a dataframe with all tickers and their data. We open the pickle file again, and make an empty dataframe.

We can then compile each ticker with their data. We drop all columns that are not *Adj Close*, and rename *Adj Close* to the ticker name since we do not have any other data than for Adjusted Close price. We choose to only include this, since the Adjusted Close price takes into account payment of dividends of companies, eventual stock splits, and Rights offerings.

For easier comparison, we also index the data by dividing the first observation in each column by the rest of the column. 

We then use the empty data frame made before, and join all the data together into a single data frame. 

We also convert the data frame to a csv file for easy access to it. 

finally, we create a data frame from the CSV file just saved, and index the date. We call this new data frame *df_stocks*.

In [6]:
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    #Iterating though all DFs

    for count, ticker in enumerate(tickers):
        df = pd.read_csv("stock_dfs/{}.csv".format(ticker))
        df.set_index("Date", inplace=True)
        df.rename(columns = {"Adj Close": ticker}, inplace=True) #Adj Close takes the categories place in the column - Simple rename
        df.drop(["Open","High","Low","Close","Volume"],1, inplace=True)
    
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how="outer")
        
        if count % 100 == 0: #Only print #100, #200, #300, etc. - It will let you know how 
            print("I have now compiled", count, "stock files")
    print(main_df.head())
    main_df.to_csv("sp500_joined_adj_closes.csv")

compile_data()

data_df = pd.read_csv("sp500_joined_adj_closes.csv")
data_df = data_df.drop(["BHGE", "DWDP", "TPR", "ARNC", "ZBH", "OKE", "EVRG", "COST", "EW", "BBT", "JNJ", "VMC", "LIN", "COTY", "DGX", "ZBH", "FTV", "LW"], axis=1)
print(data_df.head())

I have now compiled 0 stock files
I have now compiled 100 stock files
I have now compiled 200 stock files
I have now compiled 300 stock files
I have now compiled 400 stock files
I have now compiled 500 stock files
                   MMM        ABT       ABBV       ABMD        ACN       ATVI  \
Date                                                                            
2016-01-04  135.163361  39.841354  50.214832  85.239998  95.316299  36.604977   
2016-01-05  135.752548  39.832066  50.005642  85.000000  95.812393  36.137928   
2016-01-06  133.018326  39.497967  50.014351  85.300003  95.625191  35.797367   
2016-01-07  129.777802  38.551361  49.866177  81.919998  92.817101  35.291401   
2016-01-08  129.335922  37.743946  48.506435  84.580002  91.918503  34.746506   

                 ADBE   AMD         AAP       AES  ...        WLTW       WYNN  \
Date                                               ...                          
2016-01-04  91.970001  2.77  151.373932  8.105658  ...  

As we want to compare a stock from the S&P500 index by the index itself, we need the index data as a whole by itself. We get this using the same method as earlier. We appened this to the Data frame from the compile function. 

In [7]:
# a. Printing highest observed values
def plot_w(dataframe,ticker,benchmark):
#print(data_df.loc[:, I]) # 850 rows 
    max1 = data_df.loc[:, ticker].max()
    max2 = data_df.loc[:, benchmark].max()
    print("The highest stock price observed at: \n", ticker, ":", max1.round(2), " USD on the date ", "2", 
    "\n", benchmark, ":", max2.round(2), " USD on the date", "test")

# b. Setting up plot based on dropdown input
    I = data_df.columns == ticker
    plt = dataframe.loc[:, ['Date',ticker,benchmark]].plot(x='Date', y=[ticker,benchmark], style=['-bo','-ko'], figsize=(8, 5), fontsize=11, legend='true')

# c. Creating the widget for the plot
widgets.interact(plot_w, 
                 dataframe = widgets.fixed(data_df), 
                 ticker = widgets.Dropdown(
                          options=data_df.columns, value='ATVI', 
                            description='Company 1:', disabled=False,),

    benchmark = widgets.Dropdown(options=data_df.columns, value='AAPL', 
                                 description='Company 2:', disabled=False,)
    )

interactive(children=(Dropdown(description='Company 1:', index=6, options=('Date', 'MMM', 'ABT', 'ABBV', 'ABMD…

<function __main__.plot_w(dataframe, ticker, benchmark)>

In [8]:
#Get sp500 index data
data_df_indexed = data_df.set_index("Date", inplace=True)
data_df_indexed = data_df/data_df.iloc[0]*100
print(data_df_indexed)

                   MMM         ABT        ABBV        ABMD         ACN  \
Date                                                                     
2016-01-04  100.000000  100.000000  100.000000  100.000000  100.000000   
2016-01-05  100.435908   99.976686   99.583409   99.718445  100.520471   
2016-01-06   98.413006   99.138113   99.600753  100.070396  100.324070   
2016-01-07   96.015519   96.762175   99.305672   96.105115   97.377994   
2016-01-08   95.688596   94.735600   96.597824   99.225720   96.435241   
2016-01-11   95.668163   94.875352   93.525439   99.718445   97.446735   
2016-01-12   95.940615   96.575803   95.191802  103.613329   99.194744   
2016-01-13   94.483023   94.419261   89.771829  100.668700   97.476191   
2016-01-14   96.158529   96.341405   95.700480  103.097137  100.049106   
2016-01-15   94.462601   95.028730  100.576715  102.545760   97.486004   
2016-01-19   94.258256   94.770864   96.454716  101.184892   99.558083   
2016-01-20   93.284279   93.669149  10

In [9]:
data_df_indexed = data_df_indexed.reset_index()
data_df_indexed

,Date,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,WMB,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZION,ZTS
0,2016-01-04,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,2016-01-05,100.435908,99.976686,99.583409,99.718445,100.520471,98.724084,100.402300,99.277979,99.316865,...,100.342982,100.688001,101.890367,101.008395,99.805842,101.484719,99.972290,99.750734,98.914292,101.565449
2,2016-01-06,98.413006,99.138113,99.600753,100.070396,100.324070,97.793717,98.967050,90.613719,96.689422,...,87.271338,95.999261,96.611897,102.072818,98.543687,99.628818,98.725047,99.044461,96.031481,101.586625
3,2016-01-07,96.015519,96.762175,99.305672,96.105115,97.377994,96.411483,96.890290,82.310469,97.760100,...,78.544220,91.974607,87.523629,102.464978,95.825248,95.698700,96.175174,95.665412,93.148639,98.497964
4,2016-01-08,95.688596,94.735600,96.597824,99.225720,96.435241,94.922901,95.520275,77.256322,95.612181,...,77.400911,93.310829,83.961044,101.344527,93.106803,93.733637,95.260549,94.363665,92.100344,97.059433
5,2016-01-11,95.668163,94.875352,93.525439,99.718445,97.446735,96.251996,97.183860,84.476532,96.590898,...,71.227143,91.990605,82.579631,102.156848,91.262135,94.148470,94.456763,94.197479,90.265843,94.309262
6,2016-01-12,95.940615,96.575803,95.191802,103.613329,99.194744,98.670915,97.662280,86.281593,97.260888,...,63.033540,92.454702,80.878288,101.764712,92.524267,94.672473,94.983405,96.344015,89.891440,95.134310
7,2016-01-13,94.483023,94.419261,89.771829,100.668700,97.476191,92.663480,96.009570,81.227437,93.378864,...,51.867382,90.294368,76.297825,101.148441,89.514580,91.506563,92.516629,96.483577,85.698242,91.495636
8,2016-01-14,96.158529,96.341405,95.700480,103.097137,100.049106,94.098882,97.640537,79.783395,92.827104,...,69.702748,91.518542,74.887312,102.829123,89.126217,92.598266,93.043238,97.853330,86.072646,93.357288
9,2016-01-15,94.462601,95.028730,100.576715,102.545760,97.486004,92.796371,96.955526,73.285198,94.777965,...,61.356717,90.038299,84.877139,102.240878,88.349515,93.471626,91.407985,93.772010,83.639108,91.770663


In [10]:
# a. Printing highest observed values
def plot_index(dataframe_index,ticker,benchmark):
#print(data_df.loc[:, I]) # 850 rows 
    max1 = data_df_indexed.loc[:, ticker].max()
    max2 = data_df_indexed.loc[:, benchmark].max()
    print("The highest stock price observed at: \n", ticker, ":", max1.round(2), " USD on the date ", "2", 
    "\n", benchmark, ":", max2.round(2), " USD on the date", "test")

# b. Setting up plot based on dropdown input
    I = data_df_indexed.columns == ticker
    plt = dataframe_index.loc[:, ['Date',ticker,benchmark]].plot(x='Date', y=[ticker,benchmark], style=['-bo','-ko'], figsize=(8, 5), fontsize=11, legend='true')

# c. Creating the widget for the plot
widgets.interact(plot_index, 
                 dataframe_index = widgets.fixed(data_df_indexed), 
                 ticker = widgets.Dropdown(
                          options=data_df.columns, value='ATVI', 
                          description='Company 1:', disabled=False,),

                benchmark = widgets.Dropdown(options=data_df_indexed.columns, value='AAPL', 
                                 description='Company 2:', disabled=False,)
    )

interactive(children=(Dropdown(description='Company 1:', index=5, options=('MMM', 'ABT', 'ABBV', 'ABMD', 'ACN'…

<function __main__.plot_index(dataframe_index, ticker, benchmark)>

In [12]:
#pct. change on the closing prices. 
data_df_pct_change = data_df.apply(lambda x: (x - x.shift(1))/x.shift(1)*100)
data_df_pct_change = data_df_pct_change.fillna(value=0)
data_df_pct_change.head() 

data_df_pct_change = data_df_pct_change.reset_index()
data_df_pct_change.head()

,Date,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,WMB,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZION,ZTS
0,2016-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2016-01-05,0.435908,-0.023314,-0.416591,-0.281555,0.520471,-1.275916,0.402300,-0.722021,-0.683135,...,0.342982,0.688001,1.890367,1.008395,-0.194158,1.484719,-0.027710,-0.249266,-1.085708,1.565449
2,2016-01-06,-2.014122,-0.838768,0.017416,0.352945,-0.195384,-0.942392,-1.429499,-8.727273,-2.645516,...,-13.026964,-4.656702,-5.180539,1.053796,-1.264610,-1.828749,-1.247589,-0.708038,-2.914454,0.020850
3,2016-01-07,-2.436149,-2.396593,-0.296264,-3.962491,-2.936559,-1.413417,-2.098436,-9.163347,1.107337,...,-9.999982,-4.192380,-9.406986,0.384196,-2.758612,-3.944760,-2.582803,-3.411649,-3.001975,-3.040421
4,2016-01-08,-0.340489,-2.094388,-2.726781,3.247075,-0.968138,-1.543989,-1.413985,-6.140345,-2.197132,...,-1.455624,1.452817,-4.070427,-1.093496,-2.836878,-2.053386,-0.950999,-1.360730,-1.125401,-1.460468


In [26]:
# a. Printing highest observed values
def plot_pct_change(dataframe_pct_change,ticker,benchmark):
#print(data_df.loc[:, I]) # 850 rows 
    max1 = data_df_pct_change.loc[:, ticker].max()
    max2 = data_df_pct_change.loc[:, benchmark].max()
    print("The highest percentage change: \n", ticker, ":", max1.round(2), " pct. on the date ", "2", 
    "\n", benchmark, ":", max2.round(2), " pct. on the date", "test")
# b. Setting up plot based on dropdown input
    I = data_df_pct_change.columns == ticker
    plt = dataframe_pct_change.loc[:, ['Date',ticker,benchmark]].plot(x='Date', y=[ticker,benchmark], style=['-bo','-ko'], figsize=(8, 5), fontsize=11, legend='true')

# c. Creating the widget for the plot
widgets.interact(plot_pct_change, 
                 dataframe_pct_change = widgets.fixed(data_df_pct_change), 
                 ticker = widgets.Dropdown(
                          options=data_df.columns, value='ATVI', 
                          description='Company 1:', disabled=False,),

                benchmark = widgets.Dropdown(options=data_df_pct_change.columns, value='AAPL', 
                                 description='Company 2:', disabled=False)
    )

interactive(children=(Dropdown(description='Company 1:', index=5, options=('MMM', 'ABT', 'ABBV', 'ABMD', 'ACN'…

<function __main__.plot_pct_change(dataframe_pct_change, ticker, benchmark)>

In [31]:
data_df_corr = data_df.corr()


In [24]:
def correlation_table(dataframe_corr, ticker, benchmark):
    
    #max1 = data_df_pct_change.loc[:, ticker].max()
    #max2 = data_df_pct_change.loc[:, benchmark].max()
    #print("The highest percentage change: \n", ticker, ":", max1.round(2), " pct. on the date ", "2", 
    #"\n", benchmark, ":", max2.round(2), " pct. on the date", "test")

# b. Setting up plot based on dropdown input
    I = data_df_corr.columns == ticker
    plt = data_df_corr.loc[:, ['Date',ticker,benchmark]].plot(x='Date', y=[ticker,benchmark], style=['-bo','-ko'], figsize=(8, 5), fontsize=11, legend='true')

# c. Creating the widget for the plot
widgets.interact(plot_pct_change, 
                 dataframe_pct_change = widgets.fixed(data_df_pct_change), 
                 ticker = widgets.Dropdown(
                          options=data_df.columns, value='ATVI', 
                          description='Company 1:', disabled=False,),

                benchmark = widgets.Dropdown(options=data_df_pct_change.columns, value='AAPL', 
                                 description='Company 2:', disabled=False)
    
    df = pd.read_csv('sp500_joined_adj_closes.csv')
    df.set_index('Date', inplace=True) 
    df_corr = df.pct_change().corr()
    #df_corr = df.corr()

    print(df_corr.head)

    data = df_corr.values
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1) #1 by 1, plot number 1

    heatmap = ax.pcolor(data, cmap = plt.cm.RdYlGn) #red - green. Red = negative, yellow = neutral, green = positive
    fig.colorbar(heatmap)
    ax.set_xticks(np.arange(data.shape[0]) + 0.5, minor = False)
    ax.set_yticks(np.arange(data.shape[1]) + 0.5, minor = False)
    ax.invert_yaxis()
    ax.xaxis.tick_top()

    column_labels = df_corr.columns
    row_labels = df_corr.index

    ax.set_xticklabels(column_labels)
    ax.set_yticklabels(row_labels)
    plt.xticks(rotation = 90)
    heatmap.set_clim(-1,1)
    plt.tight_layout()
    plt.show()

visualize_data()

SyntaxError: invalid syntax (<ipython-input-24-6c7205234614>, line 24)

We then want to graph a single stock and compare it to the growth of the index as a whole. We start by making a copy of the data we have, using only *"ATVI"* and the *"S&P500"* data. 

On the graph above, we see the growth in the Activision Blizzard stock compared to the S&P500 index. We see that the Activision Blizzard stock has experience a wild growth since its start in 2000. The spike around 2008 is due to the merge between Activision Entertainment and Blizzard Entertainment. The rise since ca. 2013 can be due to the rise in popularity and availability to video games. The eventual drastic fall in the end of the series comes after Blizzcon 2018 where a very unpopular annoucement was made, resulting in t